In [2]:
!pip install pymysql

  Using cached PyMySQL-0.9.3-py2.py3-none-any.whl (47 kB)


In [1]:
import urllib.request
import urllib.parse
import pandas
import pymysql
import ast

#필요한 라이브러리들을 import 합니다.

# db connect 부분 입니다.
conn = pymysql.connect(host='localhost',port = 3307,
                      user = 'root', password='0313', db = 'address_svc_m',charset = 'utf8')

curs = conn.cursor(pymysql.cursors.DictCursor)
job_seq = 0

for time in range(0,2800):

    # key (mgmt_num) 과 full 도로명 주소를 쿼리 합니다.
    sql = "SELECT mgmt_num, inf_full_road_addr FROM address_svc_m.juso_info_m WHERE inf_flag = 'N' LIMIT 1"
    curs.execute(sql)
   
    #쿼리한 결과를 p_val에 저장
    p_val = curs.fetchall()
    
    #판다스에 저장 합니다.
    df = pandas.DataFrame(p_val, columns = ["mgmt_num","inf_full_road_addr"])

    p_mgmt_num = df.mgmt_num[0]
    inf_full_road_addr = df.inf_full_road_addr[0]

    # Geocoder API 2.0
    ApiKey = "9231C816-A1E4-31C9-BA64-4C02D3853BEC"
    
    address = inf_full_road_addr
    
     # API 호출 부분
    apiUrl = 'http://api.vworld.kr/req/address?service=address&request=getCoord&key='+ApiKey+'&'

    values = {
        'address':address,
        'type':'road'
    }

    param = urllib.parse.urlencode(values)
    Adding = apiUrl+param

    req = urllib.request.Request(Adding)
    res = urllib.request.urlopen(req)

    respon_data = res.read().decode()

    DataDict = ast.literal_eval(respon_data)

    v_check = DataDict['response']['status']
    
    # 위도, 경도값을 못찾은 경우를 대비해 못찾았으면 inf_flag에 'E'를 찍고 넘어감
    # 이렇게 처리 하지 않으면 프로그램이 돌다가 멈춰요..ㅠㅠ
    if v_check == 'NOT_FOUND' :
        sqlFail = "UPDATE address_svc_m.juso_info_m SET inf_flag = 'E' WHERE mgmt_num = %s"
        val = (p_mgmt_num)
        curs.execute(sqlFail,val)
        
        conn.commit()
    
    else :
        
        # 위도, 경도 데이터 받아서 update
        longtitude = DataDict['response']['result']['point']['x']
        latitude = DataDict['response']['result']['point']['y']
    
        sqlUpdate = "UPDATE address_svc_m.juso_info_m SET inf_flag = 'Y', inf_latitude = %s, inf_longtitude = %s WHERE mgmt_num = %s"
        val = (latitude,longtitude,p_mgmt_num)
        curs.execute(sqlUpdate,val)
    
    job_seq = job_seq + 1
    
    conn.commit()
        
    if job_seq%10 == 0 :
        conn.commit()
        print(job_seq)

print(job_seq)
conn.commit()

conn.close()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222